In [1]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import svm, preprocessing

from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [2]:
df = pd.read_csv('EURUSD1.csv', names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
train = np.array(df.Close)

In [3]:
def CalcReturns(data):
    R = []    
    for i in range(len(data)-1):
        v = math.log(math.fabs(data[i+1]/data[i]))
        R.append(v)
        
    return R

def ZigZag(data, minsize):
    
    N = 0
    Z = {'zigzag': [], 'time': [], 'label' : []}
    
    T = N
    
    Count = 0;
    
    Max  = data[0]
    Min  = data[0]
    
    Flag = False
    
    PriceLow = 0
    PriceHigh = 0    
        
    while N < len(data):
        
        PriceLow = data[N]
        PriceHigh = data[N]        
        
        if Flag:
            
            if PriceHigh > Max:
                
                T = N
                Max = PriceHigh                
                
            elif (Max - PriceLow >= minsize):
                
                Z['time'].append(T)
                Z['label'].append(-1)
                Z['zigzag'].append(Max)
                
                Flag = False
                Count = Count + 1                                
                
                T = N
                Min = PriceLow                
                
        else:
               
            if PriceLow < Min:
                
                T = N
                Min = PriceLow                
                    
            elif (PriceHigh - Min >= minsize):
                    
                Z['time'].append(T)
                Z['label'].append(1)
                Z['zigzag'].append(Min)
                
                Flag = True
                Count = Count + 1                
                
                T = N
                Max = PriceHigh                 
    
        N = N + 1    

    return Z

def BuildData0(zigzag, data, lag):
    
    D = []
    L = []
    
    D0 = []
    D1 = []
    
    L0 = []
    L1 = []
    
    N = len(data)
    Count = len(Z['time'])

    for i in range(0,N-lag):        
        try:
            index = zigzag["time"].index(i+lag)
            
            D0.append(data[i:i+lag])
            L0.append(zigzag["label"][index])
            
        except:
            
            D1.append(data[i:i+lag])
            L1.append(0)            

    Count0 = len(D0)
    Count1 = len(D1)
    
    #print(Count0, Count1)
    
    N = Count0 / 2
    if Count1 > N:        
        D1 = np.random.permutation(D1)
        
        D = D1[:N]
        L = L1[:N]
        
    else:        
        D = D1
        L = L1
    
    D_temp = np.concatenate((D, D0), axis=0)
    L_temp = np.concatenate((L, L0), axis=0)    
    
    I = range(len(D_temp))
    I = np.random.permutation(I)
    
    D = []
    L = []
    
    for i in I:
        D.append(D_temp[i])
        L.append(L_temp[i])
    
    D = np.array(D)
    L = np.array(L)   
    
    D = preprocessing.scale(D)
    
    return D, L
    
def BuildData1(zigzag, returns, lag):
    
    D = []
    L = []
    
    D0 = []
    D1 = []
    
    L0 = []
    L1 = []
    
    N = len(returns) + 1
    Count = len(Z['time'])

    for i in range(0,N-lag):        
        try:
            index = zigzag["time"].index(i+lag-1)
            
            D0.append(returns[i:i+lag-1])
            L0.append(zigzag["label"][index])
            
        except:
            
            D1.append(returns[i:i+lag-1])
            L1.append(0)            

    Count0 = len(D0)
    Count1 = len(D1)
    
    #print(Count0, Count1)
    
    N = Count0 / 2
    #N = Count0
    if Count1 > N:        
        D1 = np.random.permutation(D1)
        
        D = D1[:N]
        L = L1[:N]
        
    else:        
        D = D1
        L = L1
    
    D_temp = np.concatenate((D, D0), axis=0)
    L_temp = np.concatenate((L, L0), axis=0)
    
    I = range(len(D_temp))
    I = np.random.permutation(I)
    
    D = []
    L = []
    
    for i in I:
        D.append(D_temp[i])
        L.append(L_temp[i])
    
    D = np.array(D)
    L = np.array(L)   
    
    D = preprocessing.scale(D)
    
    return D, L

def BuildData2(zigzag, returns, lag):
    
    D = []
    L = []
    
    D0 = []
    D1 = []
    
    L0 = []
    L1 = []
    
    N = len(returns) + 1
    Count = len(Z['time'])

    for i in range(0,N-lag):        
        try:
            index = zigzag["time"].index(i+lag-1)
            
            D0.append(returns[i:i+lag-1])
            L0.append(zigzag["label"][index])
            
        except:
            
            D1.append(returns[i:i+lag-1])
            L1.append(0)            

    Count0 = len(D0)
    Count1 = len(D1)
    
    #print(Count0, Count1)
    
    N = Count0
    if Count1 > N:        
        D1 = np.random.permutation(D1)
        
        D = D1[:N]
        L = L1[:N]
        
    else:        
        D = D1
        L = L1
    
    D_temp = np.concatenate((D, D0), axis=0)
    L_temp = np.concatenate((L, L0), axis=0)
    
    I = range(len(D_temp))
    I = np.random.permutation(I)
    
    D = []
    L = []
    
    for i in I:
        D.append(D_temp[i])
        L.append(L_temp[i])
    
    D = np.array(D)
    L = np.array(L)   
    
    D = preprocessing.scale(D)
    
    return D, L

def BuildData3(zigzag, returns, lag):
    
    D = []
    L = []
    
    D0 = []
    D1 = []
    
    L0 = []
    L1 = []
    
    N = len(returns) + 1
    Count = len(Z['time'])

    for i in range(0,N-lag):        
        try:
            index = zigzag["time"].index(i+lag-1)
            
            D0.append(returns[i:i+lag-1])
            
            if zigzag["label"][index] == 1:                
                L0.append([0, 0, 1])
            else:
                L0.append([1, 0, 0])
            
        except:
            
            D1.append(returns[i:i+lag-1])
            L1.append([0, 1, 0])

    Count0 = len(D0)
    Count1 = len(D1)
    
    #print(Count0, Count1)
    
    N = Count0
    if Count1 > N:        
        D1 = np.random.permutation(D1)
        
        D = D1[:N]
        L = L1[:N]
        
    else:        
        D = D1
        L = L1
    
    D_temp = np.concatenate((D, D0), axis=0)
    L_temp = np.concatenate((L, L0), axis=0)
    
    I = range(len(D_temp))
    I = np.random.permutation(I)
    
    D = []
    L = []
    
    for i in I:
        D.append(D_temp[i])
        L.append(L_temp[i])
    
    D = np.array(D)
    L = np.array(L)   
    
    D = preprocessing.scale(D)
    
    return D, L

In [5]:
Z = ZigZag(train, 0.0001)
returns = CalcReturns(train)

D, L = BuildData2(Z, returns, 61)

Dim = D.shape[1]
t_count = 1024

#D, L = BuildData1(Z, returns, 61)
#D, L = BuildData0(Z, train, 61)

print(D.shape)
print(len(D))


(36576, 60)
36576


In [5]:
#clf = svm.SVC(kernel="poly", C=0.0001, degree=6, coef0=0.0001)
clf = RandomForestClassifier(n_estimators = 100, criterion='entropy') # 79%
#clf = ExtraTreesClassifier(n_estimators = 10, oob_score = True, bootstrap = True)
#clf = ExtraTreesClassifier(n_estimators = 100)
#clf = DecisionTreeClassifier()
#clf = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 100) # 71%
#clf = GradientBoostingClassifier(n_estimators = 100, learning_rate=1.1) # 81%
#clf = BaggingClassifier() #79 %
#clf = KNeighborsClassifier(n_neighbors=100, algorithm='brute') # 37%
#clf = BaggingClassifier(base_estimator = KNeighborsClassifier(n_neighbors=100)) # 36%
#clf = BaggingClassifier(base_estimator = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1)) # 81%

#clf0 = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1) # 81%
#clf1 = KNeighborsClassifier(n_neighbors=100, algorithm='brute') # 37%

#clf = VotingClassifier(estimators=[('gbc', clf0),('knc', clf1)], voting='soft')
clf.fit(D[:-t_count], L[:-t_count])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [6]:
Z = ZigZag(train, 0.0001)
returns = CalcReturns(train)

D, L = BuildData3(Z, returns, 61)
t_count = 1024

dims = D.shape[1]
print(D.shape)

(36576, 60)


In [8]:
n_nodes_hl1 = 400
n_nodes_hl2 = 400
n_nodes_hl3 = 400

n_classes = 3
batch_size = 600

hm_epochs = 6000

x_train = tf.placeholder('float', [None, 60])
y_train = tf.placeholder('float')

def neural_network_model(x):

    hl1 = {'w' : tf.Variable(tf.random_normal([dims, n_nodes_hl1])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hl2 = {'w' : tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hl3 = {'w' : tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
           'b' : tf.Variable(tf.random_normal([n_nodes_hl3]))}

    out = {'w' : tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
           'b' : tf.Variable(tf.random_normal([n_classes]))}

    l1 = tf.matmul(x, hl1['w']) + hl1['b']
    l1 = tf.nn.sigmoid(l1)

    l2 = tf.matmul(l1, hl2['w']) + hl2['b']
    l2 = tf.nn.sigmoid(l2)

    l3 = tf.matmul(l2, hl3['w']) + hl3['b']
    l3 = tf.nn.sigmoid(l3)

    y = tf.matmul(l3, out['w']) + out['b']
    return y

def train_neural_network(x,y):    
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.GradientDescentOptimizer(0.6).minimize(cost)   
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        epoch_loss = 0
        for epoch in range(hm_epochs):
            epoch_loss = 0
            #for i in range(len(D) - t_count):
            for i in range(int((len(D) - t_count)/batch_size)):
               
                #epoch_x = np.array([D[i]])
                #epoch_y = np.array([L[i]])
                
                epoch_x = D[i*batch_size : (i+1)*batch_size]
                epoch_y = L[i*batch_size : (i+1)*batch_size]
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
                
            if epoch % 100 == 0:
                print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)

        print('Last poch loss: ', epoch_loss)    
                
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        
        test_x = D[len(D)-t_count:]
        test_y = L[len(L)-t_count:]
        
        r = tf.cast(correct, 'float').eval({x:test_x, y:test_y})
        
        test_buy  = 0.0
        test_sell = 0.0
        test_hold = 0.0

        correct_buy  = 0.0
        correct_sell = 0.0
        correct_hold = 0.0

        incorrect_buy  = 0.0
        incorrect_sell = 0.0
        incorrect_hold = 0.0
        
        for i in range(len(r)):
            if L[i][2] == 1:
                test_buy += 1.0;
            elif L[i][1] == 1:
                test_hold += 1.0;
            elif L[i][0] == 1:
                test_sell += 1.0;
                
            if r[i] == 1:
                if L[i][2] == 1:
                    correct_buy += 1.0;
                elif L[i][1] == 1:
                    correct_hold += 1.0;
                elif L[i][0] == 1:
                    correct_sell += 1.0;
            else:
                if L[i][2] == 1:
                    incorrect_buy += 1.0;
                elif L[i][1] == 1:
                    incorrect_hold += 1.0;
                elif L[i][0] == 1:
                    incorrect_sell += 1.0;
        
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))
        
        print( "Test buy:  ", test_buy  )
        print( "Test sell: ", test_sell )
        print( "Test hold: ", test_hold )

        print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
        print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )
        print( "Correct hold: ", correct_hold, "Incorrect hold: ", incorrect_hold, "Accuracy: ", (correct_hold/test_hold)*100 )
        
        return prediction

train_neural_network(x_train, y_train)



Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 0, 'completed out of', 6000, 'loss', 170.25341272354126)
('Epoch', 100, 'completed out of', 6000, 'loss', 16.82700851559639)
('Epoch', 200, 'completed out of', 6000, 'loss', 4.1452061347663403)
('Epoch', 300, 'completed out of', 6000, 'loss', 1.6848427541553974)
('Epoch', 400, 'completed out of', 6000, 'loss', 0.89169360417872667)
('Epoch', 500, 'completed out of', 6000, 'loss', 0.56606864929199219)
('Epoch', 600, 'completed out of', 6000, 'loss', 0.4020091500133276)
('Epoch', 700, 'completed out of', 6000, 'loss', 0.30642509507015347)
('Epoch', 800, 'completed out of', 6000, 'loss', 0.24495655088685453)
('Epoch', 900, 'completed out of', 6000, 'loss', 0.20257644355297089)
('Epoch', 1000, 'completed out of', 6000, 'loss', 0.17181851412169635)
('Epoch', 1100, 'completed out of', 6000, 'loss', 0.1486029289662838)
('Epoch', 1200, 'completed out of', 6000, 'loss', 0.13052982650697231)
('Epoch', 1300, 'compl

<tf.Tensor 'add_11:0' shape=(?, 3) dtype=float32>

In [14]:
correct_count  = 0.0

test_buy  = 1.0
test_sell = 1.0
test_hold = 1.0

correct_buy  = 0.0
correct_sell = 0.0
correct_hold = 0.0

incorrect_buy  = 0.0
incorrect_sell = 0.0
incorrect_hold = 0.0

p = clf.predict(D[len(D)-t_count:])
c = L[len(D)-t_count:]

c0 = {'point': [], 'correct': [], 'wrong': []}

for i in range(0, t_count):
    
    if c[i] == 1:
        test_buy += 1.0;
    elif c[i] == 0:
        test_hold += 1.0;
    elif c[i] == -1:
        test_sell += 1.0;
    
    if p[i] == c[i]:
        correct_count += 1.0
        
        if c[i] == 1:
            correct_buy += 1.0;
        elif c[i] == 0:
            correct_hold += 1.0;
        elif c[i] == -1:
            correct_sell += 1.0;

    else:        
        if c[i] == 1:
            incorrect_buy += 1.0;
        elif c[i] == 0:
            incorrect_hold += 1.0;
        elif c[i] == -1:
            incorrect_sell += 1.0;
            
        c0['point'].append(D[i])
        c0['correct'].append(c[i])
        c0['wrong'].append(p[i])
         
print( "Accuracy:  ", (correct_count/t_count)*100.00 )

print( "Test buy:  ", test_buy  )
print( "Test sell: ", test_sell )
print( "Test hold: ", test_hold )

print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )
print( "Correct hold: ", correct_hold, "Incorrect hold: ", incorrect_hold, "Accuracy: ", (correct_hold/test_hold)*100 )


('Accuracy:  ', 74.21875)
('Test buy:  ', 278.0)
('Test sell: ', 256.0)
('Test hold: ', 493.0)
('Correct buy:  ', 197.0, 'Incorrect buy:  ', 80.0, 'Accuracy: ', 70.86330935251799)
('Correct sell: ', 194.0, 'Incorrect sell: ', 61.0, 'Accuracy: ', 75.78125)
('Correct hold: ', 369.0, 'Incorrect hold: ', 123.0, 'Accuracy: ', 74.84787018255578)


In [ ]:
%matplotlib qt

plt.plot(Z["time"], Z["zigzag"], range(0, len(train)), train)
#plt.plot(D[-101])



In [10]:
plt.plot(c0['point'][2])

for i in range(0, len(c0['point'])):
    print(c0['correct'][i], c0['wrong'][i])

(0, 1)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)